# Applying Decision Trees

In [1]:
import pandas

# Set index_col to False to avoid pandas thinking that the first column is row indexes (it's age)
income = pandas.read_csv("income.csv", index_col=False)

# tranformation steps omitted

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
# A list of columns to train with
# We've already converted all columns to numeric
columns = ["age", "workclass", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "hours_per_week", "native_country"]

# Instantiate the classifier
# Set random_state to 1 to make sure the results are consistent
clf = DecisionTreeClassifier(random_state=1)

# We've already loaded the variable "income," which contains all of the income data
clf.fit(income[columns], income['high_income'])

In [ ]:
import numpy
import math

# Set a random seed so the shuffle is the same every time
numpy.random.seed(1)

# Shuffle the rows  
# This permutes the index randomly using numpy.random.permutation
# Then, it reindexes the dataframe with the result
# The net effect is to put the rows into random order
income = income.reindex(numpy.random.permutation(income.index))

train_max_row = math.floor(income.shape[0] * .8)

train = income[:train_max_row]
test = income[train_max_row:]

## Evaluating error with AUC

In [ ]:
from sklearn.metrics import roc_auc_score

clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])

predictions = clf.predict(test[columns])
error = roc_auc_score(test["high_income"], predictions)
print(error)

## Compute error on the training set

In [ ]:
predictions = clf.predict(train[columns])
print(roc_auc_score(train["high_income"], predictions))

Our AUC on the training set was .947, and the AUC on the test set was .694. There's no hard and fast rule on when overfitting is occurring, but our model is predicting the training set much better than the test set. Splitting the data into training and testing sets doesn't prevent overfitting -- it just helps us detect and fix it.

Based on our AUC measurements, it appears that we are in fact overfitting. 

* The deeper a tree is, the worse it usually performs on new data, due to overfitting
* Ways to combat overfitting in decision trees:
1. "Prune" the tree after we build it to remove unnecessary leaves.
1. Use ensembling to blend the predictions of many trees.
1. Restrict the depth of the tree while we're building it.

While we'll explore all of these, we'll look at the third method first.

Limiting tree depth during the building process will result in more general rules. This prevents the tree from overfitting.

We can restrict tree depth by adding a few parameters when we initialize the DecisionTreeClassifier class:

* max_depth - Globally restricts how deep the tree can go
* min_samples_split - The minimum number of rows a node should have before it can be split; if this is set to 2, for example, then nodes with 2 rows won't be split, and will become leaves instead
* min_samples_leaf - The minimum number of rows a leaf must have
* min_weight_fraction_leaf - The fraction of input rows a leaf must have
* max_leaf_nodes - The maximum number of total leaves; this will cap the count of leaf nodes as the tree is being built

Some of these parameters aren't compatible, however. For example, we can't use max_depth and max_leaf_nodes together.

In [ ]:
# Decision trees model from the last screen
clf = DecisionTreeClassifier(random_state=1)
clf = DecisionTreeClassifier(min_samples_split=13, random_state=1)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
# 0.6995617145150872
print(train_auc)
# 0.8421431849275413

In [ ]:
# The first decision trees model we trained and tested
clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
# 0.6934656324746192
print(train_auc)
# 0.9471244501437455

clf = DecisionTreeClassifier(random_state=1, min_samples_split=13, max_depth=7)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
# 0.7436344996725136
print(train_auc)
# 0.748037708309209

## 9/15
* Set max_depth to 2 and min_samples_split to 100 when creating the DecisionTreeClassifier.
* Make predictions on the training set, compute the AUC, and assign it to train_auc.
* Make predictions on the test set, compute the AUC, and assign it to test_auc.

In [ ]:
# The first decision tree model we trained and tested
clf = DecisionTreeClassifier(random_state=1)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
# 0.6934656324746192
print(train_auc)
#0.9471244501437455

clf = DecisionTreeClassifier(random_state=1, min_samples_split=100, max_depth=2)
clf.fit(train[columns], train["high_income"])
predictions = clf.predict(test[columns])
test_auc = roc_auc_score(test["high_income"], predictions)

train_predictions = clf.predict(train[columns])
train_auc = roc_auc_score(train["high_income"], train_predictions)

print(test_auc)
# 0.6553138481876499
print(train_auc)
# 0.6624508042161483

* By artificially restricting the depth of our tree, we prevent it from creating a model that's complex enough to correctly categorize some of the rows. If we don't perform the artificial restrictions, however, the tree becomes too complex, fits quirks in the data that only exist in the training set, and doesn't generalize to new data.

* This is known as the **bias-variance tradeoff**. Imagine that we take a random sample of training data and create many models. If the models' predictions for the same row are far apart from each other, we have high variance. Imagine this time that we take a random sample of the training data and create many models. If the models' predictions for the same row are close together but far from the actual value, then we have high bias.

* High bias can cause underfitting -- if a model is consistently failing to predict the correct value, it may be that it's too simple to model the data faithfully.

* High variance can cause overfitting. If a model varies its predictions significantly based on small changes in the input data, then it's likely fitting itself to quirks in the training data, rather than making a generalizable model.

* We call this the bias-variance tradeoff because decreasing one characteristic will usually increase the other. This is a limitation of all machine learning algorithms. If you'd like to read more about the tradeoff, check out programmer [Scott Fortmann-Roe's post](http://scott.fortmann-roe.com/docs/BiasVariance.html) on this topic.

* Decision trees typically suffer from high variance. The entire structure of a decision tree can change if we make a minor alteration to its training data. By restricting the depth of the tree, we increase the bias and decrease the variance. If we restrict the depth too much, we increase bias to the point where it will underfit.